In [1]:
!pip install aiohttp~=3.6.2

     |████████████████████████████████| 1.2 MB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 149 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 256 kB 1.2 MB/s eta 0:00:01


In [2]:
!pip install PyPubSub

     |████████████████████████████████| 61 kB 596 kB/s eta 0:00:01


In [3]:
!pip install asyncio

     |████████████████████████████████| 101 kB 92 kB/s eta 0:00:01


In [4]:
!python3 -m pip install --index-url https://test.pypi.org/simple/ aries_basic_controller

Looking in indexes: https://test.pypi.org/simple/


In [5]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = "/"
ADMIN_URL = "http://researcher-agent:8051"

In [6]:
# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                           webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

In [7]:
# Spins up server to respond to any webhook events from the agent

def messages_hook(payload):
    connection_id = payload["connection_id"]
    print("Handle research messages ", payload, connection_id)

message_listener = {
    "handler": messages_hook,
    "topic": "basicmessages"
}

await agent_controller.listen_webhooks()

In [8]:
# Controllers re-emit webhooks as events through PyPubSub
# Basic state management is handled through defaults but you can choose to pass in custom handlers
custom_handler = [message_listener]
agent_controller.register_listeners(custom_handler, defaults=True)

In [13]:
# Create Invitation
response = await agent_controller.connections.accept_connection({'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '689376a9-53c1-4c07-ab9b-6a1569d29a6f', 'label': 'Bob', 'serviceEndpoint': 'http://192.168.65.3:8020', 'recipientKeys': ['27DjjAvy4v447rUJYMZzVB1sr3XN1MkU82wYkC1c4ByG']})

ACCEPTED CONNECTION -  {'their_label': 'Bob', 'initiator': 'external', 'accept': 'manual', 'my_did': 'Sckm7H5AVV7auoGGtWDAe9', 'invitation_mode': 'once', 'request_id': 'c691fa24-da80-4761-9466-6a5a5cca5df3', 'created_at': '2020-06-23 18:06:48.078332Z', 'updated_at': '2020-06-23 18:06:48.098717Z', 'connection_id': '0cafe1da-d192-4a9e-bbb7-8a616fdd3364', 'state': 'request', 'invitation_key': '27DjjAvy4v447rUJYMZzVB1sr3XN1MkU82wYkC1c4ByG', 'routing_state': 'none'}


In [14]:
print(response)

{'their_label': 'Bob', 'initiator': 'external', 'accept': 'manual', 'my_did': 'Sckm7H5AVV7auoGGtWDAe9', 'invitation_mode': 'once', 'request_id': 'c691fa24-da80-4761-9466-6a5a5cca5df3', 'created_at': '2020-06-23 18:06:48.078332Z', 'updated_at': '2020-06-23 18:06:48.098717Z', 'connection_id': '0cafe1da-d192-4a9e-bbb7-8a616fdd3364', 'state': 'request', 'invitation_key': '27DjjAvy4v447rUJYMZzVB1sr3XN1MkU82wYkC1c4ByG', 'routing_state': 'none'}


In [15]:
print("Researcher ID", response["connection_id"])

Researcher ID 0cafe1da-d192-4a9e-bbb7-8a616fdd3364


In [16]:
researcher_id = response["connection_id"]